Importamos las librerias a utilizar

In [14]:
import pandas as pd
from pandas import json_normalize
import ast

Abrimos el archivo 'australian_user_items.json'

In [15]:
# Creamos una lista vacía para extraer los datos del JSON
data_list = []

# Asignamos la ruta del json a una variable
file_path = "datasets/australian_users_items.json"

# Abrimos y procesamos cada linea del JSON
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

# Creamos un dataframe en base a la lista creada
items = pd.DataFrame(data_list)

In [16]:
# Dividimos las listas en la columna 'items' de DataFrame 'items' en filas individuales
items_exploded = items.explode(['items'])

# Normalizamos los datos de items_exploded y le damos el mismo indice del dataframe original
items_normalized = pd.json_normalize(items_exploded['items']).set_index(items_exploded['items'].index)

# Concatenamos los dataframes items_exploded y 
items = pd.concat([items_normalized, items_exploded], axis=1)

In [17]:
# Quitamos las columnas 'user_url' y 'steam_id' ya que no las vamos a utilizar
items.drop(['user_url','steam_id'], axis=1, inplace=True)

In [18]:
# Quitamos las filas donde item_id y playtime_forever estan vacías
items = items.dropna(subset=['item_id'])
items = items.dropna(subset=['playtime_forever'])

In [19]:
# Quitamos las columnas 'items' y 'playtime_2weeks' ya que no las vamos a utilizar
items.drop(['items','playtime_2weeks'], axis=1, inplace=True)

In [20]:
# Quitamos las filas del dataframe donde 'playtime_forever' es igual a 0
items = items[items['playtime_forever'] != 0]

In [21]:
# Nos quedamos con las filas que consideramos mas utiles
items = items[['user_id','item_id','playtime_forever', 'items_count', 'item_name']]

In [22]:
# Revisamos nuestro dataframe
items.head()

,user_id,item_id,playtime_forever,items_count,item_name
0,76561197970982479,10,6.0,277,Counter-Strike
0,76561197970982479,30,7.0,277,Day of Defeat
0,76561197970982479,300,4733.0,277,Day of Defeat: Source
0,76561197970982479,240,1853.0,277,Counter-Strike: Source
0,76561197970982479,3830,333.0,277,Psychonauts


Verificamos si es necesario que la columna playtime_forever sea float, checkeando si alguna fila tiene algun valor decimal

In [23]:
# Calculamos la parte decimal de los valores en la columna 'playtime_forever'
parte_decimal = items['playtime_forever'].apply(lambda x: x - int(x))

# Verificar si todos los valores de la parte decimal son cero
todos_ceros = parte_decimal.eq(0).all()

if todos_ceros:
    print("Todos los valores después de la coma son cero.")
else:
    print("Al menos un valor después de la coma no es cero.")

Todos los valores después de la coma son cero.


In [24]:
# La columna playtime_forever se refiere a minutos. Vamos a cambiarlo a horas, para que sea mas útil manejarlo a futuro y lo redondeamos a dos decimales
items['playtime_forever'] = round(items['playtime_forever']/60,2)

# Convertir la columna 'id' de float a int en el DataFrame original
items['item_id'] = items['item_id'].astype(int)

In [25]:
# Vemos el estado de nuestro dataframe
items.head()

,user_id,item_id,playtime_forever,items_count,item_name
0,76561197970982479,10,0.10,277,Counter-Strike
0,76561197970982479,30,0.12,277,Day of Defeat
0,76561197970982479,300,78.88,277,Day of Defeat: Source
0,76561197970982479,240,30.88,277,Counter-Strike: Source
0,76561197970982479,3830,5.55,277,Psychonauts


In [26]:
# Exportamos nuestro dataframe en formato parquet
items.to_parquet("datasets/Parquet/items.parquet", index=False)